<a href="https://colab.research.google.com/github/GitMishka/1/blob/main/survey2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

from google.colab import files

# Dictionary to map uploaded filenames to desired dataframe names
df_names = ["paylocity_df", "q1_df", "q2_df"]
df_dict = {}

for df_name in df_names:
    print(f"Uploading file for {df_name}")
    uploaded = files.upload()

    # Ensure only one file is uploaded at a time
    if len(uploaded) != 1:
        print("Please upload only one file at a time.")
        break

    # Read the uploaded file into a DataFrame and store in the dictionary
    uploaded_file = list(uploaded.keys())[0]
    df_dict[df_name] = pd.read_csv(uploaded_file)

# Assign the dataframes to their respective names
paylocity_df = df_dict.get("paylocity_df")
q1_df = df_dict.get("q1_df")
q2_df = df_dict.get("q2_df")

Uploading file for paylocity_df


Saving paylocity.csv to paylocity.csv
Uploading file for q1_df


Saving Qualtrix1.csv to Qualtrix1.csv
Uploading file for q2_df


Saving Qualtrix2.csv to Qualtrix2.csv


In [ ]:
q1_df['Survey Metadata - End Date (+00:00 GMT)'] = pd.to_datetime(q1_df['Survey Metadata - End Date (+00:00 GMT)'])

# Determine the most recent month
latest_month = q1_df['Survey Metadata - End Date (+00:00 GMT)'].dt.to_period('M').max()
previous_month = q1_df['Survey Metadata - End Date (+00:00 GMT)'].dt.to_period('M').max()-1

# Filter the data to only include rows from the most recent month
latest_data = q1_df[q1_df['Survey Metadata - End Date (+00:00 GMT)'].dt.to_period('M') == latest_month]
previous_data = q1_df[q1_df['Survey Metadata - End Date (+00:00 GMT)'].dt.to_period('M') == previous_month]


# Merge the dataframes on the specified columns
paylocity_df.rename(columns={'CostCenter1': 'LocCode'}, inplace=True)
merged_df = q2_df.merge(paylocity_df[['EmployeeID', 'LocCode']],
                        left_on='External Data Reference',
                        right_on='EmployeeID',
                        how='left')

# Drop the 'EmployeeID' column after merging as it's redundant
merged_df.drop('EmployeeID', axis=1, inplace=True)

# Now, group by "External Data Reference" and "CostCenter2"
grouped = merged_df.groupby(['External Data Reference', 'LocCode']).size().reset_index(name='count')

# Group by community and then aggregate to get the sum
sum_counts_by_community = merged_df.groupby('LocCode').size().reset_index(name='sum_counts')

In [ ]:


# Duplicate the column
latest_data['LocCode'] = latest_data['Embedded Data - Community']

# Now, group by 'LocCode' and aggregate
aggregated = latest_data.groupby('LocCode').agg({
    'Q1 - On a scale of 0-10 how satisfied are you working for Morning Pointe?': 'mean',
    'Q2 - What can we do better?': 'count',
    'Embedded Data - Community': 'size'
}).reset_index()

# Round the Q1 average score to 1 decimal place
aggregated['Q1 - On a scale of 0-10 how satisfied are you working for Morning Pointe?'] = aggregated['Q1 - On a scale of 0-10 how satisfied are you working for Morning Pointe?'].round(1)

# Rename the columns for clarity
aggregated.columns = ['LocCode', 'Average Q1 Score', 'Count of Q2 Entries', 'Count of Community Entries']


combined = pd.merge(aggregated, sum_counts_by_community, left_on='LocCode', right_on='LocCode', how='left')

merged_results = aggregated.merge(sum_counts_by_community, on='LocCode', how='left')

# Calculate the % of completion and round to 2 decimals
merged_results['% Completion'] = ((merged_results['Count of Community Entries'] / merged_results['sum_counts']) * 100).round(2)




<ipython-input-33-a5d24e298e3b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  latest_data['LocCode'] = latest_data['Embedded Data - Community']


In [ ]:
merged_results

In [ ]:

# Duplicate the column
previous_data['LocCode'] = previous_data['Embedded Data - Community']

# Now, group by 'LocCode' and aggregate
aggregated2 = previous_data.groupby('LocCode').agg({
    'Q1 - On a scale of 0-10 how satisfied are you working for Morning Pointe?': 'mean',
    'Q2 - What can we do better?': 'count',
    'Embedded Data - Community': 'size'
}).reset_index()

# Round the Q1 average score to 1 decimal place
aggregated2['Q1 - On a scale of 0-10 how satisfied are you working for Morning Pointe?'] = aggregated2['Q1 - On a scale of 0-10 how satisfied are you working for Morning Pointe?'].round(1)

# Rename the columns for clarity
aggregated2.columns = ['LocCode', 'Average Q1 Score', 'Count of Q2 Entries', 'Count of Community Entries']






<ipython-input-34-6f2d866690b2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  previous_data['LocCode'] = previous_data['Embedded Data - Community']


In [ ]:
final_df = pd.merge(merged_results, aggregated2, on="LocCode", how="outer", suffixes=('_latest', '_previous'))

# Calculate the "Monthly Count Variance"
final_df['Monthly Count Variance'] = final_df['Count of Community Entries_latest'] - final_df['Count of Community Entries_previous']

# Saving and downloading the combined dataframe
final_df.to_csv('final_combined_results.csv')
files.download('final_combined_results.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:

# latest_data_aggregated = latest_data.groupby('LocCode').agg({
#     'Q1 - On a scale of 0-10 how satisfied are you working for Morning Pointe?': 'mean',
#     'Q2 - What can we do better?': 'count',
#     'Embedded Data - Community': 'size'
# }).reset_index()

# latest_data_aggregated['Q1 - On a scale of 0-10 how satisfied are you working for Morning Pointe?'] = latest_data_aggregated['Q1 - On a scale of 0-10 how satisfied are you working for Morning Pointe?'].round(1)
# latest_data_aggregated.columns = ['LocCode', 'Latest Month Average Q1 Score', 'Latest Month Count of Q2 Entries', 'Latest Month Count of Community Entries']

# # Get sum_counts_by_community for latest_month
# latest_sum_counts = latest_data.groupby('LocCode').size().reset_index(name='Latest Month Sum Counts')

# # Calculate metrics for previous_month
# previous_month_aggregated = previous_month.groupby('LocCode').agg({
#     'Q1 - On a scale of 0-10 how satisfied are you working for Morning Pointe?': 'mean',
#     'Q2 - What can we do better?': 'count',
#     'Embedded Data - Community': 'size'
# }).reset_index()

# previous_month_aggregated['Q1 - On a scale of 0-10 how satisfied are you working for Morning Pointe?'] = previous_month_aggregated['Q1 - On a scale of 0-10 how satisfied are you working for Morning Pointe?'].round(1)
# previous_month_aggregated.columns = ['LocCode', 'Previous Month Average Q1 Score', 'Previous Month Count of Q2 Entries', 'Previous Month Count of Community Entries']

# # Get sum_counts_by_community for previous_month
# previous_sum_counts = previous_month.groupby('LocCode').size().reset_index(name='Previous Month Sum Counts')

# # Merge latest month's aggregated data with sum_counts
# latest_combined = pd.merge(latest_data_aggregated, latest_sum_counts, on="LocCode", how="outer")

# # Merge previous month's aggregated data with sum_counts
# previous_combined = pd.merge(previous_month_aggregated, previous_sum_counts, on="LocCode", how="outer")

# # Finally, combine results of the two months side by side
# final_combined_results = pd.merge(latest_combined, previous_combined, on="LocCode", how="outer")

# print(final_combined_results)

# # Optional: Save the combined results to CSV and download
# final_combined_results.to_csv('final_combined_results.csv')
# files.download('final_combined_results.csv')


In [ ]:
from google.colab import files
combined_results.to_csv('combined_results.csv')
files.download('combined_results.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>